In [5]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

# Root URL
root_url = "https://www.bbc.com"

# Function to fetch the webpage
def fetch_url(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None

# Function to dynamically fetch all main top-level links excluding the homepage
def get_all_main_links_excluding_homepage():
    html = fetch_url(root_url)
    if not html:
        return []
    
    soup = BeautifulSoup(html, 'html.parser')
    main_links = []
    
    try:
        # Look for main navigation container (nav, ul, div with specific classes)
        nav_container = soup.find('nav')
        
        if nav_container:
            links = nav_container.find_all('a', href=True)
            for link in links:
                href = link['href']
                # Ensure the href is a valid top-level section link
                if href.startswith("/") and len(href.split('/')) > 1 and href != "/":
                    full_link = f"https://www.bbc.com{href}"
                    if full_link and full_link not in main_links:
                        main_links.append(full_link)
    except Exception as e:
        print(f"Error finding main links: {e}")

    return main_links

# Fetch and print all main top-level links excluding the homepage
main_links = get_all_main_links_excluding_homepage()
print(main_links)

['https://www.bbc.com/news', 'https://www.bbc.com/sport', 'https://www.bbc.com/business', 'https://www.bbc.com/innovation', 'https://www.bbc.com/culture', 'https://www.bbc.com/arts', 'https://www.bbc.com/travel', 'https://www.bbc.com/future-planet', 'https://www.bbc.com/video', 'https://www.bbc.com/live']


In [184]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import logging
import time
from tqdm import tqdm  # Importing tqdm

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Function to fetch a webpage
def fetch_url(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching URL {url}: {e}")
        return None

# Function to extract all links from a page
def extract_links(html, base_url):
    soup = BeautifulSoup(html, 'html.parser')
    links = set()

    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith("http"):  # Absolute link
            links.add(href)
        elif href.startswith("/"):  # Relative link
            links.add(base_url + href)
    
    return links

# Recursive function to scrape all links and filter articles
def scrape_all_links(start_url, visited=None, depth=2):
    if visited is None:
        visited = set()

    if depth == 0 or start_url in visited:
        return set()

    visited.add(start_url)
    logging.info(f"Scraping: {start_url}")

    html = fetch_url(start_url)
    if not html:
        return set()

    # Extract all links on the current page
    base_url = "/".join(start_url.split("/")[:3])  # Base URL (e.g., "https://www.bbc.com")
    all_links = extract_links(html, base_url)

    # Filter links containing the word "article"
    article_links = {link for link in all_links if "article" in link}

    # Recursively scrape new links with tqdm progress bar
    for link in tqdm(all_links, desc="Scraping links"):
        article_links.update(scrape_all_links(link, visited, depth=depth-1))

    return article_links

# Function to categorize links by section
def categorize_links(links, sections):
    categorized_data = []
    for link in tqdm(links, desc="Categorizing links"):
        category = None
        for section in sections:
            if f"/{section}/" in link:
                category = section
                break
        categorized_data.append({'URL': link, 'Category': category})
    return categorized_data

# Main scraping from predefined `main_links`
main_links = [
    'https://www.bbc.com/news',


  'https://www.bbc.com/sport',
    'https://www.bbc.com/business',
    'https://www.bbc.com/innovation',
    'https://www.bbc.com/culture',
    'https://www.bbc.com/earth',
    'https://www.bbc.com/arts',]

sections = ["news","sport","business","innovation","culture","earth","arts"]
all_article_links = set()

# Loop through each main link and scrape articles with tqdm progress bar
for link in tqdm(main_links, desc="Starting scraping"):
    article_links = scrape_all_links(link, depth=3)
    all_article_links.update(article_links)

# Categorize the links and save them in a DataFrame with tqdm progress bar
categorized_data = categorize_links(all_article_links, sections)
df = pd.DataFrame(categorized_data)
logging.info(f"Scraping complete. Found {len(all_article_links)} article links.")

Starting scraping:   0%|          | 0/7 [00:00<?, ?it/s]


Scraping links: 100%|██████████| 221/221 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 117/117 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 128/128 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 104/104 [00:00<?, ?it/s]


Scraping links: 100%|██████████| 112/112 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 58/58 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 106/106 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 117/117 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 116/116 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 117/117 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 105/105 [00:00<?, ?it/s]


Scraping links: 100%|██████████| 117/117 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 139/139 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 105/105 [00:00<?, ?it/s]



Scraping links: 100%|██████████| 102/102 [00:00<?, ?it/s]


Scraping links: 100%|██████████

In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8671 entries, 0 to 8670
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   URL       8671 non-null   object
 1   Category  5443 non-null   object
dtypes: object(2)
memory usage: 135.6+ KB


In [187]:
df['URL'].iloc[3]

'https://www.bbc.com/news/articles/c17egnqgkwjo'

In [188]:
pattern = "https://www.bbc.co.uk"
replacement = "https://www.bbc.com"

# Create a new column 'converted_URL' where we replace the old pattern
df['converted_URL'] = df['URL'].where(~df['URL'].str.contains(pattern), df['URL'].str.replace(pattern, replacement))

In [189]:
df

,URL,Category,converted_URL
0,https://www.bbc.com/future/article/20221028-wh...,None,https://www.bbc.com/future/article/20221028-wh...
1,https://www.bbc.co.uk/news/articles/cd0elzk24dno,news,https://www.bbc.com/news/articles/cd0elzk24dno
2,https://www.nature.com//pubads.g.doubleclick.n...,None,https://www.nature.com//pubads.g.doubleclick.n...
3,https://www.bbc.com/news/articles/c17egnqgkwjo,news,https://www.bbc.com/news/articles/c17egnqgkwjo
4,https://www.dailymail.co.uk/tvshowbiz/article-...,None,https://www.dailymail.co.uk/tvshowbiz/article-...
...,...,...,...
8666,https://www.bbc.com/news/articles/cd7nz594jjyo,news,https://www.bbc.com/news/articles/cd7nz594jjyo
8667,https://www.bbc.com/news/articles/cx2l1x512dno,news,https://www.bbc.com/news/articles/cx2l1x512dno
8668,https://www.bbc.com/travel/article/20240514-a-...,None,https://www.bbc.com/travel/article/20240514-a-...
8669,https://www.bbc.com/worklife/article/20200422-...,None,https://www.bbc.com/worklife/article/20200422-...


In [190]:
df['converted_URL'].iloc[1]

'https://www.bbc.com/news/articles/cd0elzk24dno'

# Correct way to extract

In [194]:
import requests
from bs4 import BeautifulSoup
response = requests.get(df['converted_URL'].iloc[4])
response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

# Parse the content with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

In [195]:
# Find the <script> tag with `application/ld+json`
script_tag = soup.find('script', type='application/ld+json')
# Extract and parse JSON data
if script_tag:
    json_data = json.loads(script_tag.string)

    # Extract the author name
    if "author" in json_data:
        author_info = json_data["author"]
        if isinstance(author_info, list):
            # Handle multiple authors
            authors = [author.get("name", "Unknown") for author in author_info]
        else:
            # Single author
            authors = [author_info.get("name", "Unknown")]

        print(authors)
    else:
        print("No author information found.")
else:
    print("No JSON metadata found.")

['Cassie Carpenter']


In [196]:
# Extract and parse JSON data
if script_tag:
    json_data = json.loads(script_tag.string)

    # Extract the publication date
    date_published = json_data.get("datePublished", "No publication date found.")
    print("Date Published:", date_published)
else:
    print("No JSON metadata found.")

Date Published: 2025-01-19T21:52:32+0000


In [197]:
# Extract and parse JSON data
if script_tag:
    json_data = json.loads(script_tag.string)

    # Extract the headline
    headline = json_data.get("headline", "No headline found.")
    print("Headline:", headline)
else:
    print("No JSON metadata found.")

Headline: Hailey Bieber bundles up in massive faux fur coat for dinner in Aspen with BFF Kendall Jenner


In [198]:
# Extract the content from <p> tags
paragraphs = soup.find_all('p')  # Find all <p> tags

# Combine all extracted text into one document
article_content  = '\n'.join([p.get_text(strip=True) for p in paragraphs])
if article_content :
    print(article_content)
else:
    print("No content found.")

ByCASSIE CARPENTER FOR DAILYMAIL.COM
Published:08:52 AEDT, 20 January 2025|Updated:09:01 AEDT, 20 January 2025

77
Viewcomments

Hailey Baldwin Bieberand her longtime friendKendall Jennerbundled up in stylish winter coats to dine on Northern Italian fare at Casa Tua in Aspen, CO on Saturday night.
The 28-year-old IMG Model wore sunglasses at night and covered up in a massive faux-fur grey coat and glossy black high-heeled boots.
The 29-year-old Society Management Model wore a fur-collared black coat with glossy black high-heeled boots.
Hailey and Kendall not only share the same stylist (Dani Michelle) and make-up artist (Mary Phillips), their modeling careers would not exist without their famous families.
The nepo-babies were joined in theColoradomountain town by Oliver Peoples collaboratorFai Khadra, Beiber's husbandJustin Bieber, and Jenner's younger sisterKylie Jenner.
On Saturday, the 30-year-old pop star - who's been recording with Australian musician Eddie Benjamin - shared a smi

In [199]:
import requests
from bs4 import BeautifulSoup
import json

class WebPageExtractor:
    def __init__(self, url):
        self.url = url
        self.soup = None
        self.json_data = None

    def fetch_page(self):
        """Fetches the web page content and initializes BeautifulSoup."""
        try:
            response = requests.get(self.url)
            response.raise_for_status()  # Raise an HTTPError for bad responses
            self.soup = BeautifulSoup(response.content, 'html.parser')
            print("Page fetched successfully.")
        except requests.RequestException as e:
            print(f"Failed to fetch page: {e}")
            self.soup = None

    def extract_json_metadata(self):
        """Extracts JSON metadata from the <script> tag with type 'application/ld+json'."""
        if self.soup:
            script_tag = self.soup.find('script', type='application/ld+json')
            if script_tag:
                try:
                    self.json_data = json.loads(script_tag.string)
                    print("JSON metadata extracted successfully.")
                except json.JSONDecodeError as e:
                    print(f"Failed to parse JSON metadata: {e}")
                    self.json_data = None
            else:
                print("No JSON metadata found.")
        else:
            print("Soup object not initialized. Call fetch_page() first.")

    def extract_author(self):
        """Extracts author(s) from the JSON metadata."""
        if self.json_data:
            author_info = self.json_data.get("author", None)
            if author_info:
                if isinstance(author_info, list):
                    authors = [author.get("name", "Unknown") for author in author_info]
                else:
                    authors = [author_info.get("name", "Unknown")]
                return authors
            else:
                return "No author information found."
        return "No JSON metadata found."

    def extract_date(self):
        """Extracts the publication date from the JSON metadata."""
        if self.json_data:
            return self.json_data.get("datePublished", "No publication date found.")
        return "No JSON metadata found."

    def extract_headline(self):
        """Extracts the headline from the JSON metadata."""
        if self.json_data:
            return self.json_data.get("headline", "No headline found.")
        return "No JSON metadata found."

    def extract_content(self):
        """Extracts the article content from <p> tags."""
        if self.soup:
            paragraphs = self.soup.find_all('p')
            article_content = '\n'.join([p.get_text(strip=True) for p in paragraphs])
            return article_content if article_content else "No content found."
        return "Soup object not initialized. Call fetch_page() first."

In [211]:
# Example usage:
url=df['converted_URL'].iloc[326]

extractor = WebPageExtractor(url)

extractor.fetch_page()
extractor.extract_json_metadata()

print("Author(s):", extractor.extract_author())
print("Date Published:", extractor.extract_date())
print("Headline:", extractor.extract_headline())
print("Content:", extractor.extract_content())

Page fetched successfully.
JSON metadata extracted successfully.
Author(s): ['Jason Arunn Murugesu']
Date Published: 2024-11-02T07:25:18.647Z
Headline: Newcastle University research on North East accents
Content: People are being invited by researchers to help them discover how truly distinctive various north-east England accents are.
Dr Cong Zhang at the University of Newcastle is holding a drop-in session on Saturday at the Farrell Centre for members of the public between 10:00 GMT and 15:00 GMT.
People will be asked to listen to recordings of various people across the North East and try to identify where they are from solely from their voices.
Dr Zhang said the team predicted that people would struggle with the task, but said her friends were far more confident with their chances.
The researcher said the event was a celebration of the "richness and the diversity" of accents from the region.
"We want to find out using authentic audio data whether people can the differences between lo

In [216]:
df

,URL,Category,converted_URL
0,https://www.bbc.com/future/article/20221028-wh...,None,https://www.bbc.com/future/article/20221028-wh...
1,https://www.bbc.co.uk/news/articles/cd0elzk24dno,news,https://www.bbc.com/news/articles/cd0elzk24dno
2,https://www.nature.com//pubads.g.doubleclick.n...,None,https://www.nature.com//pubads.g.doubleclick.n...
3,https://www.bbc.com/news/articles/c17egnqgkwjo,news,https://www.bbc.com/news/articles/c17egnqgkwjo
4,https://www.dailymail.co.uk/tvshowbiz/article-...,None,https://www.dailymail.co.uk/tvshowbiz/article-...
...,...,...,...
8666,https://www.bbc.com/news/articles/cd7nz594jjyo,news,https://www.bbc.com/news/articles/cd7nz594jjyo
8667,https://www.bbc.com/news/articles/cx2l1x512dno,news,https://www.bbc.com/news/articles/cx2l1x512dno
8668,https://www.bbc.com/travel/article/20240514-a-...,None,https://www.bbc.com/travel/article/20240514-a-...
8669,https://www.bbc.com/worklife/article/20200422-...,None,https://www.bbc.com/worklife/article/20200422-...


In [203]:
def process_url(url):
    extractor = WebPageExtractor(url)
    extractor.fetch_page()
    extractor.extract_json_metadata()
    
    return {
        "Author(s)": extractor.extract_author(),
        "Date Published": extractor.extract_date(),
        "Headline": extractor.extract_headline(),
        "Content": extractor.extract_content(),
    }

In [210]:
df['converted_URL'].iloc[326]

'https://www.bbc.com/news/articles/crezw2zx138o'

In [245]:
df_extracted=df.copy()

In [246]:
df_extracted = df[df['converted_URL'].str.startswith('https://www.bbc.com')]

In [247]:
df_extracted=df_extracted.reset_index(drop=True)

In [248]:
df_extracted

,URL,Category,converted_URL
0,https://www.bbc.com/future/article/20221028-wh...,None,https://www.bbc.com/future/article/20221028-wh...
1,https://www.bbc.co.uk/news/articles/cd0elzk24dno,news,https://www.bbc.com/news/articles/cd0elzk24dno
2,https://www.bbc.com/news/articles/c17egnqgkwjo,news,https://www.bbc.com/news/articles/c17egnqgkwjo
3,https://www.bbc.com/news/articles/c0q04wly3nko,news,https://www.bbc.com/news/articles/c0q04wly3nko
4,https://www.bbc.com/news/articles/cjdnx79ny5vo,news,https://www.bbc.com/news/articles/cjdnx79ny5vo
...,...,...,...
5751,https://www.bbc.com/news/articles/cd7nz594jjyo,news,https://www.bbc.com/news/articles/cd7nz594jjyo
5752,https://www.bbc.com/news/articles/cx2l1x512dno,news,https://www.bbc.com/news/articles/cx2l1x512dno
5753,https://www.bbc.com/travel/article/20240514-a-...,None,https://www.bbc.com/travel/article/20240514-a-...
5754,https://www.bbc.com/worklife/article/20200422-...,None,https://www.bbc.com/worklife/article/20200422-...


# Craw historical data

In [222]:
# Use tqdm to show progress for applying the function to each URL
tqdm.pandas(desc="Extracting data from URLs")

# Apply the function to each URL and expand the results into new columns
df_content = df_extracted["converted_URL"].progress_apply(process_url)

Extracting data from URLs:   0%|          | 4/5756 [00:00<10:43,  8.93it/s]

Page fetched successfully.
JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.


Extracting data from URLs:   0%|          | 6/5756 [00:00<09:59,  9.59it/s]

Page fetched successfully.
JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.


Extracting data from URLs:   0%|          | 10/5756 [00:01<09:13, 10.37it/s]

...

Page fetched successfully.
JSON metadata extracted successfully.


Extracting data from URLs: 100%|█████████▉| 5755/5756 [1:28:02<00:00,  1.54it/s]

Page fetched successfully.
JSON metadata extracted successfully.


Extracting data from URLs: 100%|██████████| 5756/5756 [1:28:03<00:00,  1.09it/s]

Page fetched successfully.
JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.


In [239]:
# Convert the Series of dictionaries into a DataFrame
df_content=pd.DataFrame(df_content.tolist())

In [249]:
df_content

,Author(s),Date Published,Headline,Content
0,[David Robson],2022-11-07T01:00:00.000Z,Why there's more to being smart than intelligence,Multiple psychological factors contribute to a...
1,[Graham Fraser],2024-12-13T17:52:22.647Z,BBC complains to Apple over misleading shootin...,The BBC has complained to Apple after the tech...
2,[BBC News],2025-01-10T17:14:10.074Z,King's Lynn museum pulls out of LGBTQ+ history...,A museum has pulled out of a project to create...
3,[Koh Ewe],2024-12-24T04:50:35.069Z,China: Suspended death sentence for driver in ...,A Chinese court has handed a suspended death s...
4,[Gareth Gordon],2025-01-01T00:02:33.323Z,The political A to Z of Northern Ireland in 2024,"After a 24-month hiatus, devolved government i..."
...,...,...,...,...
5751,[Grace Wood],2024-11-17T11:54:12.246Z,Leeds scaffolder starts 400-mile Belle Isle to...,A scaffolder is walking 400 miles (644 km) fro...
5752,[Pauline McLean],2024-10-12T08:22:14.742Z,Meet the chihuahua taking a starring role with...,"As any performer will tell you, preparation is..."
5753,[Grace Moon],2024-05-15T13:00:00.000Z,A Korean feast that honours Buddha's birth,"Every year, millions of people celebrate the f..."
5754,[Josh Sims],2020-04-24T00:00:48.000Z,Will coronavirus change how we define heroes?,Heroism is the best of human nature – but does...


In [251]:
# Combine df_extracted and df_content by row index
df_combined = pd.concat([df_content, df_extracted],axis=1)

In [255]:
df_combined

,Author(s),Date Published,Headline,Content,URL,Category,converted_URL
0,[David Robson],2022-11-07T01:00:00.000Z,Why there's more to being smart than intelligence,Multiple psychological factors contribute to a...,https://www.bbc.com/future/article/20221028-wh...,None,https://www.bbc.com/future/article/20221028-wh...
1,[Graham Fraser],2024-12-13T17:52:22.647Z,BBC complains to Apple over misleading shootin...,The BBC has complained to Apple after the tech...,https://www.bbc.co.uk/news/articles/cd0elzk24dno,news,https://www.bbc.com/news/articles/cd0elzk24dno
2,[BBC News],2025-01-10T17:14:10.074Z,King's Lynn museum pulls out of LGBTQ+ history...,A museum has pulled out of a project to create...,https://www.bbc.com/news/articles/c17egnqgkwjo,news,https://www.bbc.com/news/articles/c17egnqgkwjo
3,[Koh Ewe],2024-12-24T04:50:35.069Z,China: Suspended death sentence for driver in ...,A Chinese court has handed a suspended death s...,https://www.bbc.com/news/articles/c0q04wly3nko,news,https://www.bbc.com/news/articles/c0q04wly3nko
4,[Gareth Gordon],2025-01-01T00:02:33.323Z,The political A to Z of Northern Ireland in 2024,"After a 24-month hiatus, devolved government i...",https://www.bbc.com/news/articles/cjdnx79ny5vo,news,https://www.bbc.com/news/articles/cjdnx79ny5vo
...,...,...,...,...,...,...,...
5751,[Grace Wood],2024-11-17T11:54:12.246Z,Leeds scaffolder starts 400-mile Belle Isle to...,A scaffolder is walking 400 miles (644 km) fro...,https://www.bbc.com/news/articles/cd7nz594jjyo,news,https://www.bbc.com/news/articles/cd7nz594jjyo
5752,[Pauline McLean],2024-10-12T08:22:14.742Z,Meet the chihuahua taking a starring role with...,"As any performer will tell you, preparation is...",https://www.bbc.com/news/articles/cx2l1x512dno,news,https://www.bbc.com/news/articles/cx2l1x512dno
5753,[Grace Moon],2024-05-15T13:00:00.000Z,A Korean feast that honours Buddha's birth,"Every year, millions of people celebrate the f...",https://www.bbc.com/travel/article/20240514-a-...,None,https://www.bbc.com/travel/article/20240514-a-...
5754,[Josh Sims],2020-04-24T00:00:48.000Z,Will coronavirus change how we define heroes?,Heroism is the best of human nature – but does...,https://www.bbc.com/worklife/article/20200422-...,None,https://www.bbc.com/worklife/article/20200422-...


In [253]:
min(df_combined['Date Published'])

'2011-05-12T12:29:18.000Z'

In [256]:
df_cleaned=df_combined[df_combined['Date Published']!='No publication date found.']

In [260]:
df_cleaned=df_cleaned[df_cleaned['Date Published']!='No JSON metadata found.']

In [267]:
df_cleaned=df_cleaned.sort_values(by="Date Published")

In [268]:
df_cleaned

,Author(s),Date Published,Headline,Content,URL,Category,converted_URL
5529,[Colleen Clark],2011-05-12T12:29:18.000Z,St Petersburg goes back to the future,"Hotels, restaurants and museums that highlight...",https://www.bbc.com/travel/article/20110511-st...,None,https://www.bbc.com/travel/article/20110511-st...
4094,[Lonely Planet Magazine],2011-07-18T15:33:29.000Z,St Petersburg's midnight sun,"During the summer months, Russia's 24-hour day...",https://www.bbc.com/travel/article/20110714-qu...,None,https://www.bbc.com/travel/article/20110714-qu...
2891,[Lonely Planet Magazine],2011-09-15T14:17:19.000Z,"Mini guide to St Petersburg, Russia",Peter the Great’s city has canals that reflect...,https://www.bbc.com/travel/article/20110901-mi...,None,https://www.bbc.com/travel/article/20110901-mi...
590,[Gabi Mocatta],2012-05-02T08:08:57.000Z,White Nights in St Petersburg,Between May and June the sun never fully sets ...,https://www.bbc.com/travel/article/20120416-wh...,None,https://www.bbc.com/travel/article/20120416-wh...
2465,[BBC World News],2012-10-31T00:00:00.000Z,St Petersburg,"Take a visual tour through St Petersburg, a ci...",https://www.bbc.com/travel/article/20121031-st...,None,https://www.bbc.com/travel/article/20121031-st...
...,...,...,...,...,...,...,...
433,[Charlotte Wright],2025-01-20T20:55:08.645Z,Eastbourne District General Hospital to get re...,The government has pledged to spend £1.5bn-£2b...,https://www.bbc.com/news/articles/c1m5eemv4mmo,news,https://www.bbc.com/news/articles/c1m5eemv4mmo
1598,[Mike Wendling],2025-01-20T21:15:20.373Z,Trump moves to make 'two genders' and anti-DEI...,Donald Trump has moved to change the US govern...,https://www.bbc.com/news/articles/czx84en1yp4o,news,https://www.bbc.com/news/articles/czx84en1yp4o
4227,[Laura Blasey],2025-01-20T21:20:29.371Z,Trump inauguration: Carrie Underwood goes a ca...,It's a rite of passage in the United States: A...,https://www.bbc.com/news/articles/cqx94ej524xo,news,https://www.bbc.com/news/articles/cqx94ej524xo
4037,[Laura Blasey],2025-01-20T21:20:29.371Z,Trump inauguration: Carrie Underwood goes a ca...,It's a rite of passage in the United States: A...,https://www.bbc.co.uk/news/articles/cqx94ej524xo,news,https://www.bbc.com/news/articles/cqx94ej524xo


In [270]:
# Splitting the DataFrame into two equal parts
mid_index = len(df_cleaned)//2

# Splitting the DataFrame
df_1 = df_cleaned.iloc[:mid_index].reset_index(drop=True)
df_2 = df_cleaned.iloc[mid_index:].reset_index(drop=True)

# Convert each DataFrame to JSON and save
df_1_json_path = "historical_data_1.json"
df_2_json_path = "historical_data_2.json"

df_1.to_json(df_1_json_path, orient="records", indent=4)
df_2.to_json(df_2_json_path, orient="records", indent=4)

In [271]:
df_cleaned['Category'].value_counts()

Category
news       2828
sport      1670
culture     526
Name: count, dtype: int64

In [272]:
import requests
import json

# URL of the JSON file
url = "https://raw.githubusercontent.com/buithehai1994/bbc-data-crawler/refs/heads/main/historical_data/historical_data_1.json"

# Fetch the JSON data
response = requests.get(url)
data = response.json()  # Parse JSON data

In [275]:
print(data[0])

{'Author(s)': ['Colleen Clark'], 'Date Published': '2011-05-12T12:29:18.000Z', 'Headline': 'St Petersburg goes back to the future', 'Content': "Hotels, restaurants and museums that highlight two different cities, one new and the other historic, both splendid.\nAchingly beautiful St Petersburg, with its baroque palaces and sweeping broad boulevards, seemed immune from the sweeping tides of change remaking the architectural and cultural character of Moscow.\nYou came here to drink in the grandeur of theWinter Palace, to float down the majestic canals, to be transported to its gilded czarist opulence. But focusing on the past is out of line with the city's history.\nFounded by the forward-thinking emperor, Peter the Great, the city has always been Russia's cultural capital. This is the city of Balanchine and Baryshnikov, Dostoevsky and Tchaikovsky. It was from here that Peter undertook the first great modernization of Russia, where citizens agitated for the October Revolution that ushered